In [2]:
from  calculate_simmilarity_resnet18 import ResnetSimmilarity

from progressbar import ProgressBar

from nearpy import Engine
from nearpy.hashes import RandomBinaryProjections
from nearpy.distances import CosineDistance

import sys


import pickle
import os
from nearpy.storage import MemoryStorage

### Form ResnetSimmilarity we can get embedding of a vector 



In [3]:
res = ResnetSimmilarity()
pbar = ProgressBar()

In [4]:
pic_one = Image.open("./data/flower_data/train/1/image_06734.jpg")
pic_one1 = Image.open("./data/flower_data/train/1/image_06734.jpg")
pic_two = Image.open("./data/flower_data/train/21/image_06774.jpg")
pic_three = Image.open("./data/download.jpeg")

In [5]:
# Dimension of our vector space
dimension = 2048

# Create a random binary hash with 10 bits
rbp = RandomBinaryProjections('rbp', 10)


msote = MemoryStorage()

engine = Engine(dimension, lshashes=[rbp],storage=msote,distance=CosineDistance())

#### Adding the Images to the Hash Table

In [6]:
%%time

rootdir = 'data/flower_data/train/'


for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if '.jpg' in file:
        
            img_path = os.path.join(subdir, file)
            img = Image.open(img_path)
            
            img_emb = res.getMapping(img)
            img_emb = img_emb.view(-1,2048)
            img_emb = img_emb.numpy()
            
            
            engine.store_vector(img_emb[0],file)

CPU times: user 4min 15s, sys: 3min 6s, total: 7min 21s
Wall time: 2min 12s


#### Save Image Hashed Table to File

In [7]:
%%time
filehandler = open("hashed_object.pkl", 'w')
pickle.dump(engine, filehandler)

CPU times: user 7.6 s, sys: 208 ms, total: 7.81 s
Wall time: 7.81 s


### Restore Hased Table 

In [8]:
%%time
filehandler = open("hashed_object.pkl", 'r')
en_loaded = pickle.load(filehandler)

CPU times: user 1.94 s, sys: 212 ms, total: 2.15 s
Wall time: 3 s


In [9]:

# image to test query
pic_one1_emb = res.getMapping(pic_one1)
pic_one1_emb = pic_one1_emb.view(-1,2048)
pic_one1_emb = pic_one1_emb.numpy()
print(pic_one1_emb.shape)


query = pic_one1_emb[0]

(1, 2048)


#### Query on one image and get simmilarity result

In [10]:
%%time

N = en_loaded.neighbours(query)
for i in range(len(N)):
    print(N[i][1],N[i][2])

('image_07800.jpg', 0.0036786446068788337)
('image_02532.jpg', 0.003726762434540287)
('image_07612.jpg', 0.0037849506654052956)
('image_07481.jpg', 0.0037967772461857674)
('image_01767.jpg', 0.003798351456365201)
('image_01630.jpg', 0.0038020903802128414)
('image_07126.jpg', 0.003818604502385803)
('image_07479.jpg', 0.0038275743743392265)
('image_00458.jpg', 0.0038363795060631922)
('image_05497.jpg', 0.003839477365755517)
CPU times: user 96 ms, sys: 116 ms, total: 212 ms
Wall time: 70 ms
